<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Transform" data-toc-modified-id="Transform-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Transform</a></span><ul class="toc-item"><li><span><a href="#Export--" data-toc-modified-id="Export---1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Export -</a></span></li></ul></li></ul></div>

In [ ]:
#default_exp transforms

In [ ]:
#export
from mantisshrimp.imports import *
from mantisshrimp.core import *
from mantisshrimp.data.all import *

# Transform
> Trasformers

In [ ]:
#export
class Tfm:
    def __init__(self, tfms): self.tfms = tfms
    def __call__(self, item):
        tfmed = self.apply(**item.asdict())
        return item.replace(**tfmed)

In [ ]:
#export
class AlbuTfm(Tfm):
    def __init__(self, tfms):
        self.bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels'])
        super().__init__(tfms=A.Compose(tfms, bbox_params=self.bbox_params))

    def apply(self, img, labels, bboxes=None, masks=None, iscrowds=None, **kwargs):
        # Substitue labels with list of idxs, so we can also filter out iscrowd in case any bbox is removed 
        # TODO: Same should be done if a mask is completely removed from the image (if bbox is not given)
        d = self.tfms(image=img, labels=list(range_of(labels)),
                      masks=masks.data if masks else None,
                      bboxes=lmap(lambda o: o.xyxy, bboxes))
        return {
            'img': d['image'],
            'labels': [labels[i] for i in d['labels']],
            'bboxes': lmap(lambda o: BBox.from_xyxy(*o), d['bboxes']),
            'masks': ifnotnone(d['masks'], lambda o: Mask(np.stack(o))),
            'iscrowds': lmap(iscrowds.__getitem__, d['labels']),
        }

In [ ]:
catmap,train_rs,valid_rs = test_utils.sample_records()


Removed 0 images that don't have annotations


In [ ]:
tfm = AlbuTfm([A.HorizontalFlip(p=1.)])
ds = Dataset(train_rs)
tfm_ds = Dataset(train_rs, tfm=tfm)

In [ ]:
item = ds[0]
tfmed = tfm_ds[0]
test_eq(tfmed.img, item.img[:,::-1,:])

In [ ]:
tfm = AlbuTfm([A.CenterCrop(100, 100, p=1.)])
tfm_ds = Dataset(train_rs, tfm=tfm)
tfmed = tfm_ds[0]
test_eq(len(tfmed.bboxes), 2)
test_eq(len(tfmed.iscrowds), 2)

## Export -

In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 02_data.core.ipynb.
Converted 04_data.annotations.ipynb.
Converted 06_data.load.ipynb.
Converted 07_transforms.ipynb.
Converted 08_models.ipynb.
Converted 11_metrics.core.ipynb.
Converted Untitled.ipynb.
Converted Untitled1.ipynb.
Converted data_refactor.ipynb.
Converted index.ipynb.
